In [1]:
import argparse
import sys
import time
from datetime import date, timedelta
import os
from datetime import datetime
import numpy as np
from google2pandas import GoogleAnalyticsQueryV4
import pandas as pd
import urllib
from sqlalchemy import create_engine

In [2]:
#Параметры для подключение к sql ХД
params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=***********;Database=USERDB;Trusted_Connection=yes;")
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [3]:
#Параметры для GA
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/analytics']
CLIENT_SECRETS_PATH = 'C://Users//kiril//ga_api//client_secret_sysuevks.json' # Path to client_secrets.json file.
conn = GoogleAnalyticsQueryV4(
                              secrets='C://Users//kiril//ga_api//token_for_ga_api.json'
                             )

In [ ]:
#Указываем в какой таймдельте нам нужны выгрузки, выгрузка для ДК
max_date_query_dk = '''
SELECT max(date) as max_date FROM USERDB.SYSV.ga_report_dk
'''
temp_max_date = pd.read_sql(max_date_query_dk, engine).max_date.iloc[0]
max_date_dk = datetime.strptime(temp_max_date, '%Y-%m-%d')  + timedelta(days=1)
current_date = date.today() - timedelta(days=1)
dr_dk = pd.date_range(max_date_dk, current_date)

In [8]:
for i in dr_dk:
    print(i)

2021-02-28 00:00:00
2021-03-01 00:00:00
2021-03-02 00:00:00
2021-03-03 00:00:00
2021-03-04 00:00:00
2021-03-05 00:00:00


In [ ]:
%%time
#vb - views+bounces
base_data_for_vb_dk = []
for i in dr_dk:
    query_vb_dk = {
        'reportRequests': [{
            'viewId' : '169135929',

            'dateRanges': [{
                'startDate' : i.strftime('%Y-%m-%d'),
                'endDate'   : i.strftime('%Y-%m-%d')}],

            'dimensions' : [
                {'name' : 'ga:date'},
            {'name': 'ga:pagePath'}],

            'metrics'   : [
                {'expression' : 'ga:pageviews'},
                {'expression': 'ga:bounces'}],

            "dimensionFilterClauses": [
                {'operator': 'AND',
                 'filters': [
                  {"dimensionName": 'ga:medium',
                  "not": True,
                  "operator": 'IN_LIST',
                  "expressions": [
                    'Display', 'OLV'
                  ],
                  "caseSensitive": False
                }      
                 ]

        }]


        }
        ]}
    df_vb_dk = conn.execute_query(query_vb_dk)
    base_data_for_vb_dk.append(df_vb_dk)


In [8]:
vb_df_dk = pd.concat(base_data_for_vb_dk)

In [9]:
vb_df_dk['pageviews'] = vb_df_dk['pageviews'].astype('float')
vb_df_dk['bounces'] = vb_df_dk['bounces'].astype('float')
vb_df_dk['date'] = vb_df_dk['date'].apply(lambda x:datetime.strftime(x, "%Y-%m-%d"))
vb_df_dk['date'] = vb_df_dk['date'].astype('str')

In [10]:
vb_df_dk['pagePath'] = vb_df_dk['pagePath'].apply(lambda x: x.split('?')[0])
vb_df_dk['pagePath'] = vb_df_dk['pagePath'].str.replace(',', '')
vb_df_dk['pagePath'] = vb_df_dk['pagePath'].apply(lambda x: x if x.endswith('/') else x + '/')

In [11]:
vb_df_dk_clean = vb_df_dk.groupby(['pagePath', 'date'], as_index=False).agg({'pageviews':'sum','bounces':'sum'})

In [ ]:
%%time
base_data_for_cr_dk = []
for i in dr_dk:
    query_conv_dk = {
        'reportRequests': [{
            'viewId' : '169135929',

            'dateRanges': [{
                'startDate' : i.strftime('%Y-%m-%d'),
                'endDate'   : i.strftime('%Y-%m-%d')}],

            'dimensions' : [
                {'name' : 'ga:date'},
            {'name': 'ga:goalCompletionLocation'}],

            'metrics'   : [
                {'expression' : 'ga:goal11Completions'}]

        }]
    }
    temp_conv_df_dk = conn.execute_query(query_conv_dk)
    base_data_for_cr_dk.append(temp_conv_df_dk)

In [ ]:
conv_df_dk = pd.concat(base_data_for_cr_dk)

In [ ]:
conv_df_dk = conv_df_dk.rename(columns={'goalCompletionLocation':'pagePath', 'goal11Completions':'conversion'})
conv_df_dk['conversion'] = conv_df_dk['conversion'].astype('float')
conv_df_dk['date'] = conv_df_dk['date'].apply(lambda x:datetime.strftime(x, "%Y-%m-%d"))
conv_df_dk['date'] = conv_df_dk['date'].astype('str')

In [ ]:
conv_df_dk['pagePath'] = conv_df_dk['pagePath'].apply(lambda x: x.split('?')[0])
conv_df_dk['pagePath'] = conv_df_dk['pagePath'].str.replace(',', '')
conv_df_dk['pagePath'] = conv_df_dk['pagePath'].apply(lambda x: x if x.endswith('/') else x + '/')

In [ ]:
conv_df_dk_clean = conv_df_dk.groupby(['date', 'pagePath'], as_index=False).conversion.sum()

In [ ]:
dk_df_for_deploy = vb_df_dk_clean.merge(conv_df_dk_clean, how='left', on=['date', 'pagePath']).sort_values(by='pageviews', ascending=False)

In [ ]:
with engine.begin() as connection:
    dk_df_for_deploy.to_sql("ga_report_dk", connection, schema="SYSV", if_exists='append', index=False, chunksize=30, method='multi') 

In [ ]:
#Указываем в какой таймдельте нам нужны выгрузки, выгрузка для Халвы
max_date_query_halva = '''
SELECT max(date) as max_date FROM USERDB.SYSV.ga_report_halva
'''
temp_max_date = pd.read_sql(max_date_query_halva, engine).max_date.iloc[0]
max_date_halva = datetime.strptime(temp_max_date, '%Y-%m-%d') + timedelta(days=1)
current_date = date.today() - timedelta(days=1)
dr_halva = pd.date_range(max_date_halva, current_date)

In [ ]:
for i in dr_halva:
    print(i)

In [ ]:
%%time
#vb - views+bounces
base_data_for_vb_halva = []
for i in dr_halva:
    query_vb_halva = {
        'reportRequests': [{
            'viewId' : '138955743',

            'dateRanges': [{
                'startDate' : i.strftime('%Y-%m-%d'),
                'endDate'   : i.strftime('%Y-%m-%d')}],

            'dimensions' : [
                {'name' : 'ga:date'},
            {'name': 'ga:pagePath'}],

            'metrics'   : [
                {'expression' : 'ga:pageviews'},
                {'expression': 'ga:bounces'}],

            "dimensionFilterClauses": [
                {'operator': 'AND',
                 'filters': [
                  {"dimensionName": 'ga:medium',
                  "not": True,
                  "operator": 'IN_LIST',
                  "expressions": [
                    'Display', 'OLV'
                  ],
                  "caseSensitive": False
                }      
                 ]

        }]


        }
        ]}
    df_vb_halva = conn.execute_query(query_vb_halva)
    base_data_for_vb_halva.append(df_vb_halva)

In [ ]:
# %%time
# #vb - views+bounces
# base_data_for_vb_halva = []
# for i in dr_halva:
#     query_vb_halva = {
#     'reportRequests': [{
#         'viewId' : '138955743',

#         'dateRanges': [{
#             'startDate' : i.strftime('%Y-%m-%d'),
#             'endDate'   : i.strftime('%Y-%m-%d')}],

#         'dimensions' : [
#             {'name' : 'ga:date'},
#         {'name': 'ga:pagePath'}],

#         'metrics'   : [
#             {'expression' : 'ga:pageviews'},
#             {'expression': 'ga:bounces'}]
        
#     }]
#     }
#     df_vb_halva = conn.execute_query(query_vb_halva)
#     base_data_for_vb_halva.append(df_vb_halva)

In [ ]:
vb_df_halva = pd.concat(base_data_for_vb_halva)

In [ ]:
vb_df_halva['pageviews'] = vb_df_halva['pageviews'].astype('float')
vb_df_halva['bounces'] = vb_df_halva['bounces'].astype('float')
vb_df_halva['date'] = vb_df_halva['date'].apply(lambda x:datetime.strftime(x, "%Y-%m-%d"))
vb_df_halva['date'] = vb_df_halva['date'].astype('str')

In [ ]:
vb_df_halva['pagePath'] = vb_df_halva['pagePath'].apply(lambda x: x.split('?')[0])
vb_df_halva['pagePath'] = vb_df_halva['pagePath'].str.replace(',', '')
vb_df_halva['pagePath'] = vb_df_halva['pagePath'].apply(lambda x: x if x.endswith('/') else x + '/')

In [ ]:
vb_df_halva_clean = vb_df_halva.groupby(['pagePath', 'date'], as_index=False).agg({'pageviews':'sum','bounces':'sum'})

In [ ]:
%%time
base_data_for_conv_halva = []
for i in dr_halva:
    query_conv_halva = {
        'reportRequests': [{
            'viewId' : '138955743',

            'dateRanges': [{
                'startDate' :  i.strftime('%Y-%m-%d'),
                'endDate'   :  i.strftime('%Y-%m-%d')}],

            'dimensions' : [
                {'name' : 'ga:date'},
            {'name': 'ga:goalCompletionLocation'}],

            'metrics'   : [
                {'expression' : 'ga:goal5Completions'}]

        }]
    }
    temp_conv_df_halva = conn.execute_query(query_conv_halva)
    base_data_for_conv_halva.append(temp_conv_df_halva)

In [ ]:
conv_df_halva = pd.concat(base_data_for_conv_halva)

In [ ]:
conv_df_halva = conv_df_halva.rename(columns={'goalCompletionLocation':'pagePath', 'goal5Completions':'conversion'})
conv_df_halva['conversion'] = conv_df_halva['conversion'].astype('float')
conv_df_halva['date'] = conv_df_halva['date'].apply(lambda x:datetime.strftime(x, "%Y-%m-%d"))
conv_df_halva['date'] = conv_df_halva['date'].astype('str')

In [ ]:
conv_df_halva['pagePath'] = conv_df_halva['pagePath'].apply(lambda x: x.split('?')[0])
conv_df_halva['pagePath'] = conv_df_halva['pagePath'].str.replace(',', '')
conv_df_halva['pagePath'] = conv_df_halva['pagePath'].apply(lambda x: x if x.endswith('/') else x + '/')

In [ ]:
conv_df_halva_clean = conv_df_halva.groupby(['date', 'pagePath'], as_index=False).conversion.sum()

In [ ]:
halva_df_for_deploy = vb_df_halva_clean.merge(conv_df_halva_clean, how='left', on=['date', 'pagePath']).sort_values(by='pageviews', ascending=False)

In [ ]:
%%time
with engine.begin() as connection:
    halva_df_for_deploy.to_sql("ga_report_halva", connection, schema="SYSV", if_exists='append', index=False, chunksize=30, method='multi') 